In [1]:
class GromacsITPFile(object):
    def __init__(self, filename):
        self.filename = filename
    def read_residue(self, residue_name):
        tables = {}
        with open(self.filename, "r") as file_handle:
            sectionline = 0
            in_table = False
            correct_section = False
            for line in file_handle:
                stripline = line.strip()
                if len(stripline) > 0:
                    if stripline[0] == "[" and stripline[-1] == "]":
                        table_name = stripline[1:-1].strip()
                        sectionline = 0
                        in_table = True
                    elif line[0] == ";" and sectionline == 1:
                        table_columns = line[1:].split()
                    elif in_table:
                        if table_name == "moleculetype":
                            if stripline.split()[0] == residue_name:
                                correct_section = True
                            else:
                                correct_section = False
                        if table_name in ["atoms", "bonds", "angles"] and correct_section:
                            if table_name not in tables:
                                tables[table_name] = []
                            tables[table_name].append({key: value for key, value in zip(table_columns, stripline.split())})
                        
                sectionline += 1
        rp = ResidueParameters(residue_name)
        for atom in tables["atoms"]:
            rp.add_atom(atom["id"], atom)
        for bond in tables["bonds"]:
            rp.add_bond(bond["i"], bond["j"], bond)
        for angle in tables["angles"]:
            rp.add_angle(angle["i"], angle["j"], angle["k"], angle)
        return rp

In [2]:
class RPAtom(object):
    def __init__(self, attrs):
        self.attrs = attrs
        self.i_bonded = []
        self.j_bonded = []
        self.i_angled = []
        self.j_angled = []
        self.k_angled = []
    def bonded(self):
        ba = []
        for bond in self.i_bonded:
            ba.append(bond.j_atom)
        for bond in self.j_bonded:
            ba.append(bond.i_atom)
        return ba

class RPBond(object):
    def __init__(self, i_atom, j_atom, attrs):
        i_atom.i_bonded.append(self)
        j_atom.j_bonded.append(self)
        self.attrs = attrs
        self.i_atom = i_atom
        self.j_atom = j_atom

class RPAngle(object):
    def __init__(self, i_atom, j_atom, k_atom, attrs):
        i_atom.i_angled.append(self)
        j_atom.j_angled.append(self)
        k_atom.k_angled.append(self)
        self.attrs = attrs
        self.i_atom = i_atom
        self.j_atom = j_atom
        self.k_atom = k_atom

class ResidueParameters(object):
    def __init__(self, resname):
        self.resname = resname
        self.atoms = {}
        self.bonds = []
        self.angles = []
    def add_atom(self, id, attributes):
        self.atoms[id] = RPAtom(attributes)
    def add_bond(self, i, j, attributes):
        i_atom = self.atoms[i]
        j_atom = self.atoms[j]
        self.bonds.append(RPBond(i_atom, j_atom, attributes))
    def add_angle(self, i, j, k, attributes):
        i_atom = self.atoms[i]
        j_atom = self.atoms[j]
        k_atom = self.atoms[k]
        self.angles.append(RPAngle(i_atom, j_atom, k_atom, attributes))

In [3]:
dlpg_itp = GromacsITPFile("data/DLPG.itp")
dlpg = dlpg_itp.read_residue("DLPG")
dvpe_itp = GromacsITPFile("data/DVPE.itp")
dvpe = dvpe_itp.read_residue("DVPE")

In [20]:
from itertools import product

def martini_type_similarity(type1, type2):
    if type1 == type2:
        return 1
    else:
        differences = {
            "C1.C3" : 0.6
        }
        return getattr(differences, ".".join(sorted([type1, type2])), 0)
        

class ExchangeMap(object):
    def __init__(self, from_residue, to_residue):
        self.from_residue = from_residue
        self.to_residue = to_residue
    def _new_martini_lipid(self):
        from_atoms = {
            k:(
                atom.attrs["type"], 
                [x.attrs["type"] for x in atom.bonded()]
            ) for k, atom in self.from_residue.atoms.items()
        }
        to_atoms = {
            k:(
                atom.attrs["type"], 
                [x.attrs["type"] for x in atom.bonded()]
            ) for k, atom in self.to_residue.atoms.items()
        }
        from_by_type = {}
        grow_maps = list()
        done_maps = list()
        for from_atom_id, (from_atom_type, from_connected_types) in from_atoms.items():
            for to_atom_id, (to_atom_type, to_connected_types) in from_atoms.items():
                if martini_type_similarity(from_atom_type, to_atom_type) > 0.5:
                    # Atom types are similar
                    candidate_map = {from_atom_id:to_atom_id}
                    connected_similarity = 0
                    for fc_type, tc_type in product(from_connected_types, to_connected_types):
                        similarity = martini_type_similarity(fc_type, tc_type)
                        connected_similarity += similarity                           
                    if connected_similarity > 1:
                        grow_maps.append(candidate_map)
        iters = 0
        while len(grow_maps) > 0 and iters < 1000:
            iters += 1
            a_b_map = grow_maps.pop()
            grown = False
            b_a_map = {v:k for k, v in a_b_map.items()}
            for a_atom, b_atom in a_b_map.items():
                new_map = {x:y for x, y in a_b_map.items()}
                a_bonded_atoms = [atom for atom in self.from_residue.atoms[a_atom].bonded() 
                            if atom.attrs["id"] not in a_b_map]
                b_bonded_atoms = [atom for atom in self.to_residue.atoms[b_atom].bonded() 
                            if atom.attrs["id"] not in b_a_map]
                for a_bonded_atom in a_bonded_atoms:
                    for b_bonded_atom in b_bonded_atoms:
                        if martini_type_similarity(a_bonded_atom.attrs["type"], b_bonded_atom.attrs["type"]) > 0.5:
                            new_map[a_bonded_atom.attrs["id"]] = b_bonded_atom.attrs["id"]
                            if new_map not in grow_maps:
                                grow_maps.append(new_map)
                                grown = True
            if not grown:
                done_maps.append(a_b_map)
                if len(done_maps) > 10:
                    done_maps = sorted(done_maps, key=lambda x : -len(x))[:10]
        direct_overlay = sorted(done_maps, key=lambda x : -len(x))[0]
        al
        
                        
    def new(self, scheme="martini-lipid"):
        if scheme == "martini-lipid":
            self._new_martini_lipid()
            
x = ExchangeMap(dlpg, dvpe)
x.new()

['GL0', 'C3A']
['D3B', 'C4B', 'NH3', 'D3A', 'C4A']


In [13]:
a = [{1:2, 2:5}]
{1:2, 2:5} in a

True